In [2]:
import cv2
import numpy as np

# Open input video file
cap = cv2.VideoCapture("D://DBDA//task_2//task_2_video.mp4")

# Define green color range in HSV
green_lower = np.array([50, 50, 50])
green_upper = np.array([70, 255, 255])

# Create output video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('task_2_output.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS),
                      (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# Set minimum contour area
min_area = 100

while cap.isOpened():
    # Read current frame from input video
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame from BGR to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Apply green color range mask on HSV frame
    green_mask = cv2.inRange(hsv, green_lower, green_upper)
    green_frame = cv2.bitwise_and(hsv, hsv, mask=green_mask)

    # Find contours in the green mask
    contours, hierarchy = cv2.findContours(green_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw red dot at centroid of each contour with area greater than min_area
    for cnt in contours:
        # Calculate contour area
        area = cv2.contourArea(cnt)

        if area > min_area:
            # Calculate centroid of contour using moments
            M = cv2.moments(cnt)
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])

            # Draw red dot at centroid position
            cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)

    # Merge original frame with red dots
    output_frame = cv2.addWeighted(frame, 0.7, cv2.cvtColor(green_frame, cv2.COLOR_HSV2BGR), 0.3, 0)

    # Write resulting frame to output video
    out.write(output_frame)

    # Display resulting frame
    cv2.imshow('Task 2 Output', output_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release input and output video files
cap.release()
out.release()
cv2.destroyAllWindows()
